# H2O model interpretation
Needs to be executed with papermill

In [1]:
import pandas as pd
import seaborn as sns
import h2o
from models.h2o import H2o
import os
import glob

In [ ]:
# loads model paths and exp_ids
reps = list(range(0, 3))

exp_id = '2022102713559613'

models = []
exp_paths = []
for rep in reps:
    exp_path = os.path.join('experiments', exp_id + '_' + str(rep))
    exp_paths.append(exp_path)
    models.append(H2o.load(os.path.join(exp_path, 'fold_0')))

In [ ]:
# load base models
path = 'output'

df_models = []
# loop over 30 bootrap models
for idx, bootstrap_model in enumerate(models):
    
    # loop over base models
    base_models = []
    base_models_imp = []
    if 'base_models' in bootstrap_model.actual_params:
        
        # get metalearner
        metalearner = h2o.get_model(bootstrap_model.metalearner()['name']).varimp()
        metalearner = pd.DataFrame(metalearner, columns=['model', 'relimp', 'scaimp', 'perc']).set_index('model')
        
        for base_model in bootstrap_model.actual_params['base_models']:
            base_models.append(base_model['name'])
            base_models_imp.append(metalearner.loc[base_model['name'], 'perc'])
    else:
        base_models.append(bootstrap_model_name.split('/')[1])
        base_models_imp.append(1)
            
    df_base_models = pd.DataFrame(base_models, columns=['base_model'])
    df_base_models['rep'] = reps[idx]
    df_base_models['path'] = glob.glob(os.path.join(exp_paths[idx], 'fold_0', '*_AutoML_*'))[0]
    df_base_models['importance'] = base_models_imp
    df_models.append(df_base_models)
    
df_models = pd.concat(df_models, ignore_index=True)
df_models['base_model_type'] = df_models['base_model'].str.split('_').str[0]
df_models['model_type'] = df_models['path'].str.split('/').str[-1].str.split('_').str[0:2].str.join(' ')
df_models